# Food Delivery Optimization in Milan

## Introduction

In this project we will find the best placements for a number, in this case 10, of food couriers in Milan to optimize the delivery service with faster and better deliveries and also improve working conditions of couriers who will have to travel less distance.  

## Business Problem

In Milan there are many food delivery services and many more restaurants, so the optimal organization beetween these two fundamental components of the food business is essential to create an efficient and profitable system. 
By finding the optimal location in which the couriers will wait the next order is useful to reduce the time necessary to reach the restaurant, so the meal will be better, the client will wait less and the courier will travel less distance.

## Data Description

Using the Foursquare API we will retrieve the data of the restaurants and food services in the area of Milan, which has a location of 45°28'01" N and 09°11'24" E. For each activity we will retrieve name, position and type of cuisine. Through a KK-means algorithm with 10 point we will define the locations which minimize the distance beetween the couriers and the restaurants and finally we will analize the composition of each cluster of restaurants by their cousine.    

In [16]:
import numpy as np
import requests
import json

In [33]:
!conda install pandas==1.1.0 --yes
import pandas as pd
from pandas.io.json import json_normalize

Solving environment: ...working... done

# All requested packages already installed.



In [34]:
CLIENT_ID = 'TS4O02SW4CRHYCFOSATBLREBBH5QB3FIZD4IEDUPFZFUOLUV' # your Foursquare ID
CLIENT_SECRET = 'NGJ0VOEPCK4LCVQMXDUYRNSO2BO0ZW0OHQGV1TWRQQU4K4KY' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: TS4O02SW4CRHYCFOSATBLREBBH5QB3FIZD4IEDUPFZFUOLUV
CLIENT_SECRET:NGJ0VOEPCK4LCVQMXDUYRNSO2BO0ZW0OHQGV1TWRQQU4K4KY


In [35]:
milan_latitude = 45.4669444
milan_longitude = 9.19

In [81]:
# type your answer here
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
categoryId = '4d4b7105d754a06374d81259' #Food category

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    milan_latitude, 
    milan_longitude, 
    radius, 
    LIMIT, 
    categoryId)
url # display URL


'https://api.foursquare.com/v2/venues/explore?&client_id=TS4O02SW4CRHYCFOSATBLREBBH5QB3FIZD4IEDUPFZFUOLUV&client_secret=NGJ0VOEPCK4LCVQMXDUYRNSO2BO0ZW0OHQGV1TWRQQU4K4KY&v=20180605&ll=45.4669444,9.19&radius=500&limit=100&categoryId=4d4b7105d754a06374d81259'

In [82]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f3bee622002af58d5880dad'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Duomo',
  'headerFullLocation': 'Duomo, Milan',
  'headerLocationGranularity': 'neighborhood',
  'query': 'food',
  'totalResults': 84,
  'suggestedBounds': {'ne': {'lat': 45.471444404500005,
    'lng': 9.196404492045794},
   'sw': {'lat': 45.4624443955, 'lng': 9.183595507954205}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b55fd64f964a520b7fa27e3',
       'name': 'Luini',
       'location': {'address': 'Via Santa Radegonda 16',
        'crossStreet': 'Via San Raffaele',
        'lat': 45.465707270425064,
        'lng': 9.191430566038733,
        'labeledLatLngs': 

In [123]:
# function that extracts the category of the venue
def get_category_type(row):

    categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [135]:
venues = json_normalize(results['response']['groups'][0]['items'])  

filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
venues = venues.loc[:, filtered_columns]

categories=[]

for x in range(0, len(venues)):

    categories.append(get_category_type(venues.iloc[x]))
    
venues['venue.categories'] = categories
venues.columns = ['Name', 'Category', 'Latitude', 'Longitude']
venues

ImportError: cannot import name 'is_url' from 'pandas.io.common' (C:\Users\danif\Anaconda3\lib\site-packages\pandas\io\common.py)

                               Name                    Category   Latitude  \
0                             Luini                      Bakery  45.465707   
1                          Spontini                 Pizza Place  45.465655   
2   Ristorante Trussardi alla Scala          Italian Restaurant  45.467036   
3      Galleria Ristorante Pizzeria          Italian Restaurant  45.465881   
4                  VUN Andrea Aprea          Italian Restaurant  45.465290   
5                     Panini Durini              Sandwich Place  45.465238   
6                       Cracco Cafè                        Café  45.465320   
7                 Voce Aimo e Nadia                  Restaurant  45.467320   
8                             Obicà          Italian Restaurant  45.464897   
9                     Spazio Milano          Italian Restaurant  45.464788   
10              Ristorante Victoria          Italian Restaurant  45.466445   
11       La Locanda Del Gatto Rosso          Italian Restaurant 